In [ ]:
import cv2 
import numpy as np
import uuid
import smtplib
import datetime

In [ ]:
camera = cv2.VideoCapture(1)
h = int(camera.get(cv2.CAP_PROP_FRAME_HEIGHT))
w = int(camera.get(cv2.CAP_PROP_FRAME_WIDTH))

In [ ]:
num = 0

while True: 
    ret, frame = camera.read()
    
    cv2.imshow('FAU Library Mask Check', frame)
    
    if cv2.waitKey(1) % 256 == 13:
        store_images = './mask-check-web-application/frontend/public/Photos'+str(num)+'.jpg'.format(str(uuid.uuid4()))
        date_time = str(datetime.datetime.now())
        cv2.imwrite(store_images, frame)
        camera.release()
        cv2.destroyAllWindows()
        break
        
    num += 1

In [ ]:
import json
from ibm_watson import VisualRecognitionV4
from ibm_watson.visual_recognition_v4 import FileWithMetadata, AnalyzeEnums
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
key = 'Nx3NvwU5bvQsnmHGODdCcR_1LoRnQZFulZZg5kxdm469'
link = 'https://api.us-south.visual-recognition.watson.cloud.ibm.com/instances/c89018d4-7929-4a8e-a664-5a42ad47936f' 
collection_id = '8c34763e-4cfc-47f8-932b-1feca74a0b3c'

In [ ]:
authenticator = IAMAuthenticator(key)
service = VisualRecognitionV4('2018-03-19', authenticator=authenticator)
service.set_service_url(link)

In [ ]:
filepath1 = './mask-check-web-application/frontend/public/Photos' + str(num) + '.jpg'
filename = 'student'+str(num)+'.jpg'

In [ ]:
with open(filepath1, 'rb') as image:
    student_label = service.analyze(collection_ids=[collection_id], features=[AnalyzeEnums.Features.OBJECTS.value], images_file=[FileWithMetadata(image)]).get_result()

In [ ]:
student_label

In [ ]:
text_style = cv2.FONT_HERSHEY_PLAIN
face = student_label['images'][0]['objects']['collections'][0]['objects'][0]['object']
coordinates = student_label['images'][0]['objects']['collections'][0]['objects'][0]['location']

In [ ]:
from matplotlib import pyplot as plt
text_style = cv2.FONT_HERSHEY_PLAIN
face = student_label['images'][0]['objects']['collections'][0]['objects'][0]['object']
coordinates = student_label['images'][0]['objects']['collections'][0]['objects'][0]['location']

In [ ]:
student = cv2.imread(filepath1)
student = cv2.cvtColor(student, cv2.COLOR_BGR2RGB)

if face == 'YesMask':
    student = cv2.putText(student, text = face, org=(coordinates['left']+coordinates['width'], coordinates['top']+coordinates['height']), fontFace = text_style, fontScale = 2, color = (0,255,0), thickness=3)
    student = cv2.putText(student, text = face, org=(coordinates['left']+coordinates['width'], coordinates['top']+coordinates['height']), fontFace = text_style, fontScale = 2, color = (0,255,0), thickness=3)
    student = cv2.rectangle(student, (coordinates['left'], coordinates['top']), (coordinates['left']+coordinates['width'], coordinates['top']+coordinates['height']),(0,255,0), 5)
    subject = "Type your desired subject line."
    message = "Type your desired body of text."
    email = 'Subject: {}\n\n{}'.format(subject, message)
    mail = smtplib.SMTP('smtp.gmail.com', 587)
    mail.ehlo()
    mail.starttls()
    mail.login('your email username', 'your email password')
    mail.sendmail('the sender email address', 'the recipient email address', email)
    mail.close
    
elif face == 'NoMask':
    student = cv2.putText(student, text = face, org=(coordinates['left']+coordinates['width'], coordinates['top']+coordinates['height']), fontFace = text_style, fontScale = 2, color = (255,0,0), thickness=3)
    student = cv2.rectangle(student, (coordinates['left'], coordinates['top']), (coordinates['left']+coordinates['width'], coordinates['top']+coordinates['height']),(255,0,0), 5)
    subject = "Type your desired subject line."
    message = "Type your desired body of text."
    email = 'Subject: {}\n\n{}'.format(subject, message)
    mail = smtplib.SMTP('smtp.gmail.com', 587)
    mail.ehlo()
    mail.starttls()
    mail.login('your email username', 'your email password')
    mail.sendmail('the sender email address', 'the recipient email address', email)
    mail.close

In [ ]:
plt.imshow(student)

In [ ]:
from pymongo import MongoClient
from pprint import pprint

In [ ]:
client = MongoClient("mongodb+srv://dbUser:<password>@masklog.nyfeh.mongodb.net/MaskLog?retryWrites=true&w=majority")

In [ ]:
db = client.get_database('MaskLog')
records = db.LogRow

In [ ]:
records.count_documents({})

In [ ]:
if face == 'YesMask':
    new_entry = {'time': date_time, 'result': 'yes', 'photoname': filename}
    records.insert_one(new_entry)
    
if face == 'NoMask':
    new_entry = {'time': date_time, 'result': 'no', 'photoname': filename}
    records.insert_one(new_entry)

In [ ]:
records.count_documents({})